In [26]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import scipy
from sklearn.tree import export_graphviz
from subprocess import call
from IPython.display import Image
from sklearn.model_selection import train_test_split
from scipy import stats

%matplotlib inline
matplotlib.style.use('bmh')

In [38]:
data_all_thomas = pd.read_excel('annotations/data_all_thomas.xlsx')
data_labels_video_features = pd.read_csv('annotations/combined_labels_video_features.csv')
labels = data_labels_video_features[['gold_gt_max_aro','gold_gt_max_like','gold_gt_max_val','gold_gt_min_aro','gold_gt_min_like','gold_gt_min_val']]
labels_max = labels[['gold_gt_max_aro','gold_gt_max_like','gold_gt_max_val']]
labels_min = labels[['gold_gt_min_aro','gold_gt_min_like','gold_gt_min_val']]
video_features = data_labels_video_features.drop(['gold_gt_max_aro','gold_gt_max_like','gold_gt_max_val','gold_gt_min_aro','gold_gt_min_like','gold_gt_min_val'],axis=1)

#Applying z normalization
video_features.apply(stats.zscore)
#features = list(video_features.columns)


,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,...,Unnamed: 16374,Unnamed: 16375,Unnamed: 16376,Unnamed: 16377,Unnamed: 16378,Unnamed: 16379,Unnamed: 16380,Unnamed: 16381,Unnamed: 16382,Unnamed: 16383
0,0.063630,-0.142438,0.449996,0.820350,1.071412,1.261822,-0.462493,0.724469,0.527336,-0.501534,...,-0.073761,-0.409489,1.522084,0.528014,-0.699945,0.615996,0.310785,-0.355818,-0.222007,0.203132
1,0.153056,-0.798246,-1.155813,-0.204897,1.050355,-0.299558,-0.222933,-1.819520,0.767132,0.393520,...,-0.124390,0.818038,1.126642,0.016784,1.032239,-0.786948,0.335472,1.147176,0.568247,0.200026
2,-0.055316,-0.637399,-1.200148,0.175554,0.933502,-0.494045,-0.092673,-1.867703,0.664002,0.346203,...,0.254428,0.970204,-1.135148,-0.105556,0.182156,-1.181622,0.652101,-0.258536,-0.751698,-0.691265
3,0.943994,-0.458596,-1.492402,-0.564439,-0.644132,-0.446328,-0.905464,0.777686,-0.241621,0.284834,...,0.052561,0.407138,-0.544393,0.643058,-0.419786,-0.578819,-0.148596,1.172315,-0.593892,0.311826
4,-0.650914,0.897160,0.968321,0.775981,0.814335,-0.562671,0.813696,-0.673415,-0.362649,-0.053585,...,1.923727,-0.464431,-0.164953,1.270190,-0.338140,-0.218062,0.066068,0.090702,0.488732,-0.018006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,0.269881,-1.512857,-0.823172,-0.854453,0.061847,1.425443,-0.667509,1.182807,0.805094,0.952008,...,-1.981705,2.087464,1.408979,0.327324,-0.658321,-0.161019,0.558722,-1.216203,1.539183,1.580753
956,0.071536,0.477311,0.518995,-0.379994,-0.897044,-0.506778,-0.096523,-0.661873,-0.593999,-1.626915,...,-1.530072,-0.114678,-0.856914,-0.128381,-0.651918,-1.013577,-1.596719,-1.354767,-0.606125,-1.051897
957,-0.696776,1.662495,1.769843,1.679851,0.878894,-0.159759,2.579271,0.956634,0.682997,-0.432261,...,-0.482469,0.397617,0.591116,-0.088669,0.201367,0.167362,-0.021622,0.494654,-0.186531,-0.160217
958,-0.695812,0.049083,-2.324831,-0.935711,0.231920,-0.357998,-0.955622,-0.456385,0.670117,-0.062161,...,-0.217182,0.382917,-0.186142,-0.562966,0.163105,-0.373773,-0.607977,-1.130881,0.079659,-0.507262


In [3]:
#Random forest
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(n_estimators=100, bootstrap=True, max_features='sqrt')
rf.fit(video_features,labels_max)

In [4]:
#Decision tree
from sklearn.tree import DecisionTreeClassifier


tree = DecisionTreeClassifier(random_state=50)
tree.fit(video_features, labels_max)
print(f'Decision tree has {tree.tree_.node_count} nodes with maximum depth {tree.tree_.max_depth}.')
export_graphviz(tree, 'tree.dot', rounded = True, feature_names = features, class_names = ['0', '1','2'], filled = True)
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=400']);

In [37]:
#Support Vector Machine
from sklearn.metrics import classification_report, confusion_matrix,recall_score
from sklearn.svm import SVC
from mpl_toolkits import mplot3d

X_train, X_test, y_train, y_test = train_test_split(video_features, labels['gold_gt_min_val'], test_size=0.30)

clf = SVC(kernel='linear')
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
report = classification_report(y_test,y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_csv('report.csv')



In [42]:
#Logistic regression
from sklearn.linear_model import LogisticRegression



X_train, X_test, y_train, y_test = train_test_split(video_features, labels['gold_gt_max_val'], test_size=0.30)
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test,y_pred))

precision    recall  f1-score   support

           1       0.00      0.00      0.00         6
           2       0.74      0.71      0.73       173
           3       0.60      0.64      0.62       109

    accuracy                           0.67       288
   macro avg       0.45      0.45      0.45       288
weighted avg       0.67      0.67      0.67       288



In [6]:
#Boxplots
def save_boxplot(x,y):
    box_data_1 = data_all_thomas[data_all_thomas[x] == 1 ].get(y)
    box_data_2 = data_all_thomas[data_all_thomas[x] == 2 ].get(y)
    box_data_3 = data_all_thomas[data_all_thomas[x] == 3 ].get(y)
    box_data = [box_data_1,box_data_2,box_data_3]
    fig, ax = plt.subplots()
    ax.boxplot(box_data)
    ax.set_ylabel(y)
    ax.set_xlabel(x)
    fig.savefig('boxplots/boxplot_'+x+'_'+y+'.svg')


def create_boxplots():
    classifications = ['agreeableness','conscientiousness','extraversion','interview','neuroticism','openness']
    variables = ['valence','arousal','likeability']
    for x in variables:
        for y in classifications:
            save_boxplot(x,y)

In [7]:
#Basis statistics
def stats(x):
    data_class_1 = data_all_thomas[data_all_thomas[x] == 1 ].get('interview')
    data_class_2 = data_all_thomas[data_all_thomas[x] == 2 ].get('interview')
    data_class_3 = data_all_thomas[data_all_thomas[x] == 3 ].get('interview')
    print('data1: mean=%.3f stdv=%.3f' % (np.mean(data_class_1), np.std(data_class_1)))
    print('data2: mean=%.3f stdv=%.3f' % (np.mean(data_class_2), np.std(data_class_2)))
    print('data3: mean=%.3f stdv=%.3f' % (np.mean(data_class_3), np.std(data_class_3)))    
    stat, p = ttest_ind(data_class_2, data_class_3) 
    print(stat, p)

In [8]:
#Statistical significance
def mannWithNeyuScore():
    data_class_1 = data_all_thomas[data_all_thomas['likeability'] == 1 ].get('interview')
    data_class_2 = data_all_thomas[data_all_thomas['likeability'] == 2 ].get('interview')
    data_class_3 = data_all_thomas[data_all_thomas['likeability'] == 3 ].get('interview')
    scipy.stats.mannwhitneyu(data_class_1, data_class_3)